<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Lionshead Fire Climate Data Analysis

_by Joe Hamman (CarbonPlan), September 17, 2020_

This notebook performs a brief analysis of the climatic conditions leading up to
the Lionshead Fire. Below we show trends in summer temperature, 1000-hr dead
fuel moisture, and burning index for the region of the fire.

## Methodology

For the purpose of this analysis, we have simply extracted a daily timeseries of
climate variables from the GridMET dataset at the point nearest the centroid of
the fire. We aggregate these to seasonal average and plot the trends for only
the summer season.

## References

- GridMET dataset. Available on-line
  [http://www.climatologylab.org/gridmet.html].
- Abatzoglou, J. T. (2013), Development of gridded surface meteorological data
  for ecological applications and modelling. Int. J. Climatol., 33: 121–131.


In [ ]:
import warnings
import xarray as xr
import matplotlib.pyplot as plt
from carbonplan.styles import colors
import seaborn as sns

# gridmet's opendap server is issuing a bunch of warnings
warnings.filterwarnings("ignore")

# plot styles
c = colors(mode="light")

### Load the GridMET dataset


In [ ]:
def load_gridmet(var):
    """helper function to load data from opendap server by year/var"""

    ds_list = []

    for year in range(1979, 2021):
        url = f"http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/{var}/{var}_{year}.nc"
        ds = (
            xr.open_dataset(url, engine="pydap")
            .sel(lon=-121.2683, lat=44.7603, method="nearest")
            .load()
        )
        if "day" in ds.coords:
            ds = ds.rename({"day": "time"})

        ds_list.append(ds)

    return xr.concat(ds_list, dim="time")


temp = []
for var in ["fm1000", "bi", "tmmx", "tmmn"]:
    ds = load_gridmet(var)
    if "air_temperature" in ds:
        ds = ds.rename({"air_temperature": var})
    temp.append(ds)

ds = xr.merge(temp)

ds["tmean"] = (ds["tmmx"] + ds["tmmn"]) / 2

display(ds)

### Plot trends in climate variables


In [ ]:
jja = ds["time.season"] == "JJA"

for var in ["dead_fuel_moisture_1000hr", "burning_index_g", "tmean"]:
    plt.figure()
    data = ds[var].where(jja).resample(time="AS").mean()
    sns.regplot(
        x=data["time.year"].values,
        y=data.values,
        robust=True,
        scatter=False,
        color=c["secondary"],
    )
    sns.lineplot(x=data["time.year"].values, y=data.values, color=c["primary"])
    plt.title("jja " + var)